# AA1 lab 08  
# Warm-up: boosting, bagging with trees with 'diabetes' dataset 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
import joblib

In [ ]:
# dataset

diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

print(X.shape)
print(y.shape)

print(diabetes.DESCR)

## Random Forests and Boosting

We are going to make a simple comparison between bagging and boosting using trees

In [ ]:
# ti figure out how many cores and run in parallel random forests

N_CORES = joblib.cpu_count(only_physical_cores=True)
print(f"Number of physical cores: {N_CORES}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate

In [ ]:
from tqdm import tqdm
import time

# we'll place results here..

results = {k + "-crossval": [] for k in ["rf", "gb"]}

T_vals = np.geomspace(1, 300, num=50, dtype=int)

rf_time = 0
gb_time = 0

for T in tqdm(T_vals, total=len(T_vals)):
    rfc = RandomForestRegressor(
        n_estimators=T, n_jobs=N_CORES, criterion="squared_error"
    )
    res = cross_validate(rfc, X, y, cv=5)
    results["rf-crossval"].append(np.mean(res["test_score"]))
    rf_time += np.mean(res["fit_time"])

    gbc = GradientBoostingRegressor(n_estimators=T, loss="squared_error")
    res = cross_validate(gbc, X, y, cv=5)
    results["gb-crossval"].append(np.mean(res["test_score"]))
    gb_time += np.mean(res["fit_time"])

In [ ]:
import matplotlib.pyplot as plt

for k in results:
    plt.plot(np.array(T_vals), np.array(results[k]), label=k)
plt.legend()

print(f"time for random forests: {rf_time:.2f}s")
print(f"time for gradient boosting: {gb_time:.2f}s")